# Milestone Report Part 2: CNN

This is the code a regular CNN is trained and tested.

In [1]:
import tensorflow as tf
import numpy as np

import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math

IMG_SIZE_PX = 50
SLICE_COUNT = 20

n_classes = 2

x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.8 # was 0.8


In [2]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,64])), # was 32, 64
               #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,64,64])),
               #                                  64 features
               'W_conv3':tf.Variable(tf.random_normal([3,3,3,64,32])),
               #                                  64 features
               'W_conv4':tf.Variable(tf.random_normal([3,3,3,32,32])),
               'W_fc':tf.Variable(tf.random_normal([1024,1024])),
               'W_fc2':tf.Variable(tf.random_normal([1024,1024])),
               'W_fc3':tf.Variable(tf.random_normal([1024,1024])),# was 54080, 1024
               'out':tf.Variable(tf.random_normal([1024, n_classes]))} # was 1024

    biases = {'b_conv1':tf.Variable(tf.random_normal([64])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_conv3':tf.Variable(tf.random_normal([32])),
                'b_conv4':tf.Variable(tf.random_normal([32])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
              'b_fc2':tf.Variable(tf.random_normal([1024])),
              'b_fc3':tf.Variable(tf.random_normal([1024])),# was 16
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.leaky_relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.leaky_relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    
    conv3 = tf.nn.leaky_relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    conv3 = maxpool3d(conv3)
    
    conv4 = tf.nn.leaky_relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    conv4 = maxpool3d(conv4)
    print(conv4.get_shape)
    fc = tf.reshape(conv4,[-1, 1024])# was 54080
    fc = tf.nn.leaky_relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.leaky_relu(tf.matmul(fc, weights['W_fc2'])+biases['b_fc2'])
    fc = tf.nn.leaky_relu(tf.matmul(fc, weights['W_fc3'])+biases['b_fc3'])
    #fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output


In the above you have to absolutely use leak_relu instead of relu, because negative input kill too many neuros if you use relu as you active function. leak_relu does not appear in tensorflow until the newest version (1.4.0), so update if you cannot run it.

Also I am totally overfitting here, you will see how learning error drop to 0 at the training stage. I think doing this in this case may do more good than harm given that how few the training samples I used. But if you want to re-run it in the full data set with all 1500 samples, you should definitely avoid overfitting. Just uncomment the dropout layer to prevent overfit (sort of)

In [3]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [4]:
#much_data = np.load('muchdata-50-50-20.npy')



#train_data = much_data[:-30]
#validation_data = much_data[-30:]
#train_data = much_data
# If you are working with the basic sample data, use maybe 2 instead of 100 here... you don't have enough data to really do this
much_data = np.load('muchdata-50-50-20-b.npy')
train_data = [[item[0],item[1]] for item in much_data]
#test_data = train_data
test_data = np.load('testdata-50-50-20-b.npy')
test_data = [[item[0],item[1],item[2]] for item in test_data]

P_list = []
proba_list = []
proba_list_2 = []

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=tf.to_int32(y) ))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
    
    hm_epochs = 30
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        successful_runs = 0
        total_runs = 0
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for data in train_data:
                total_runs += 1
                try:
                    X = data[0]
                    Y = data[1]
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_runs += 1
                except Exception as e:
                    # I am passing for the sake of notebook space, but we are getting 1 shaping issue from one 
                    # input tensor. Not sure why, will have to look into it. Guessing it's
                    # one of the depths that doesn't come to 20.
                    pass
                    #print(str(e))
            
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

            #correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            #accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            #for i in xrange(10):
            #    val_data = chunkIt(validation_data,10)[i]
            #    print('Accuracy:',accuracy.eval({x:[i[0] for i in val_data], y:[i[1] for i in val_data]}))
            #print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
            
            #print accuracy
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            predictions = tf.argmax(prediction,1)
            proba = tf.cast(prediction, 'float')
            proba_2 = tf.nn.softmax(logits=prediction)
                        
        print('Done. Finishing accuracy:')
        for j in xrange(30):
            te_data = chunkIt(test_data,30)[j]
            print('Accuracy:',accuracy.eval({x:[i[0] for i in te_data], y:[i[1] for i in te_data]}))
            P_list.append(predictions.eval({x: [i[0] for i in te_data]}))
            proba_list.append(proba.eval({x: [i[0] for i in te_data]}))
            proba_list_2.append((proba_2.eval({x: [i[0] for i in te_data]})))
            
        #print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        print('fitment percent:',successful_runs/total_runs)

# Run this locally:
train_neural_network(x)

<bound method Tensor.get_shape of <tf.Tensor 'MaxPool3D_3:0' shape=(?, 4, 4, 2, 32) dtype=float32>>
Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 1, 'completed out of', 30, 'loss:', 30150386647040.0)
('Epoch', 2, 'completed out of', 30, 'loss:', 17376966311936.0)
('Epoch', 3, 'completed out of', 30, 'loss:', 12069842911232.0)
('Epoch', 4, 'completed out of', 30, 'loss:', 9354998382592.0)
('Epoch', 5, 'completed out of', 30, 'loss:', 7071894634496.0)
('Epoch', 6, 'completed out of', 30, 'loss:', 6180944117760.0)
('Epoch', 7, 'completed out of', 30, 'loss:', 4353802043392.0)
('Epoch', 8, 'completed out of', 30, 'loss:', 3503065235456.0)
('Epoch', 9, 'completed out of', 30, 'loss:', 2891192041472.0)
('Epoch', 10, 'completed out of', 30, 'loss:', 2143011995648.0)
('Epoch', 11, 'completed out of', 30, 'loss:', 1039085076480.0)
('Epoch', 12, 'completed out of', 30, 'loss:', 1729846149120.0)
('Epoch', 13, 'completed out of', 30, 'loss:', 1201292705792.0)


Finishing accuarcy looks pretty good... Let us save it and submit!

In [5]:
feature_test = [np.sum(img[0]) for img in test_data]

In [6]:
labels = pd.read_csv('../../cap2input/stage2_solution.csv', index_col=0)
labels.cancer.value_counts()

0    353
1    153
Name: cancer, dtype: int64

In [7]:
proba_list

[array([[ -5.39881701e+11,  -1.30290483e+12],
        [ -7.92488968e+11,  -1.31879757e+12],
        [ -9.36451113e+11,  -1.06559596e+12],
        [ -7.08738286e+11,  -1.42710525e+12],
        [ -7.12453521e+11,  -1.38000230e+12],
        [ -3.09675491e+11,  -1.42660914e+12],
        [ -1.62929928e+12,  -1.72067770e+12],
        [ -1.24044024e+12,  -1.66145216e+12],
        [ -1.51368211e+12,  -2.32912532e+12],
        [ -1.09853422e+12,  -1.04029775e+12],
        [ -7.87655295e+11,  -1.27330288e+12],
        [ -1.05392256e+12,  -1.64442944e+12],
        [ -7.27473717e+11,  -1.19627409e+12],
        [ -5.64556333e+11,  -1.08911788e+12],
        [ -7.90516204e+11,  -1.49522376e+12],
        [ -1.57472562e+12,  -1.71016061e+12]], dtype=float32),
 array([[ -1.93715778e+12,  -2.75862598e+12],
        [ -1.48820918e+11,  -1.57085978e+12],
        [ -1.41426360e+12,  -1.67797064e+12],
        [ -3.72131627e+11,  -9.71210359e+11],
        [ -5.82091538e+11,  -1.33414728e+12],
        [ -8.6815

In [8]:
P_list[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [9]:
ori_list = [item for sublist in P_list for item in sublist]
print ori_list

true_list = [item[1][1] for item in test_data]
print true_list

patient_list = [item[2] for item in test_data]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [10]:
ori_list_proba = [item for sublist in proba_list for item in sublist]

ori_list_proba_true = [[-sublist[1],-sublist[0]] if (sublist[0]+sublist[1]<0) else sublist for sublist in ori_list_proba]


proba_list_true = [sublist/sum(sublist) for sublist in ori_list_proba_true]

proba_list_0 = [sublist[0] for sublist in proba_list_true]
proba_list_1 = [0.2 if sublist[1]<0.2 else 0.8 if sublist[1]>0.8 else sublist[1] for sublist in proba_list_true]

#ori_list_proba = [np.exp(item) for sublist in proba_list for item in sublist]

print proba_list_1

[0.29297027, 0.37535831, 0.46774679, 0.33183062, 0.34048676, 0.2, 0.48636132, 0.42745906, 0.39390007, 0.51361406, 0.38217917, 0.39058009, 0.3781544, 0.34139514, 0.34584695, 0.47938514, 0.41253129, 0.2, 0.45735976, 0.27701929, 0.30376774, 0.34011167, 0.5734154, 0.23123626, 0.43945813, 0.41907007, 0.42448258, 0.42866811, 0.46431336, 0.35289991, 0.4039835, 0.2, 0.47761407, 0.45549899, 0.27624455, 0.3338328, 0.37333888, 0.52827704, 0.41414568, 0.36546957, 0.3798807, 0.38892716, 0.50489831, 0.49633345, 0.26168594, 0.41890615, 0.20917745, 0.34401399, 0.41630152, 0.4613153, 0.30596164, 0.48323569, 0.46963671, 0.3138395, 0.49285245, 0.40828276, 0.2, 0.33849093, 0.31936127, 0.28820145, 0.56317866, 0.230967, 0.42384729, 0.29910782, 0.22171678, 0.34731296, 0.35820422, 0.46197227, 0.48129684, 0.31205228, 0.28003445, 0.46266955, 0.37077317, 0.75854754, 0.2, 0.43331894, 0.40886107, 0.57173026, 0.4982675, 0.41537589, 0.42656198, 0.51544988, 0.35175443, 0.27855065, 0.2, 0.46086046, 0.40707105, 0.40951

In [11]:
#ori_list_proba_2 = [item for sublist in proba_list_2 for item in sublist]
#print ori_list_proba_2

In [12]:
labels['proba'] = proba_list_1
labels['predictions'] = ori_list
labels['true'] = true_list
labels['feature'] = feature_test
labels['patient'] = patient_list


In [13]:
pred = labels[['patient','proba']].reset_index(drop=True)
pred.columns = ['id','cancer']

In [14]:
#pred[pred == 0] = 0.4
#pred[pred == 1] = 0.6

In [15]:
pred

,id,cancer
0,7a632e7b8309e4b57a5964762a9e594a,0.292970
1,d662c2884bf723e2b0b2401ccab08887,0.375358
2,edde23221a8c4038f9a10daa3088ee94,0.467747
3,91541739b4a106e5481a6156c5b3cfbb,0.331831
4,d82af7a99fb96b2df37b4a0db270795f,0.340487
5,2b76a32e5c12e4312cb029721a24b190,0.200000
6,307a08ece9a33b772c903f5afbf9a799,0.486361
7,5d669e1f9abdff1a282b924c828a9b0a,0.427459
8,ad690e1d0df849ee7983f69ab43e785c,0.393900
9,c6c77fd425da262d6eeed53cff38cd91,0.513614


In [16]:
pred.to_csv('../../cap2input/result/new_fifth_sub.csv', index=False)

That is about it! The result is not good since we only have 300 samples (actually fewer than that, probably only 270 of them due to unlabelled data). The result constantly pass the brenchmark, but only in 1 of the 4 times it will get close to a bronze medal. On average, it ranked around 220 out of 394 teams